In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET

def get_urals_prices_cbr():
    # Формируем даты для запроса
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*10)  # за последние 10 лет
    
    # Формируем URL запроса
    url = "http://www.cbr.ru/scripts/XML_dynamic.asp"
    params = {
        'date_req1': start_date.strftime('%d/%m/%Y'),
        'date_req2': end_date.strftime('%d/%m/%Y'),
        'VAL_NM_RQ': 'R01239'  # Код для нефти Urals
    }
    
    try:
        # Выполняем запрос
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        # Парсим XML
        root = ET.fromstring(response.content)
        
        # Собираем данные
        data = []
        for record in root.findall('Record'):
            date = record.get('Date')
            value = record.find('Value').text
            nominal = record.find('Nominal').text
            
            data.append({
                'Date': datetime.strptime(date, '%d.%m.%Y'),
                'Price': float(value.replace(',', '.')),
                'Nominal': int(nominal)
            })
        
        # Создаем DataFrame
        df = pd.DataFrame(data)
        
        # Сортируем по дате
        df = df.sort_values('Date')
        
        # Сохраняем в CSV
        df.to_csv('../../../data/external/macro/urals_prices_cbr.csv', index=False)
        
        print(f"Данные успешно загружены и сохранены в 'urals_prices_cbr.csv'")
        print(f"Количество записей: {len(df)}")
        print("\nПервые 5 записей:")
        print(df.head())
        
        return df
        
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к API ЦБ: {e}")
        return None

if __name__ == "__main__":
    get_urals_prices_cbr()

In [ ]:
df = pd.read_csv('urals_prices_cbr.csv')
df[-10:]